In [1]:
import pandas as pd
import numpy as np
import requests
import re
import urllib.parse
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup

F1 API Key- Gotten from ergast.com
This is a free API and does not require a private API key

In [8]:
f1_base = "https://ergast.com/api/f1"
# This code, when added to the f1_base api, lets us choose which driver
# Before adding results, we retrieve information on Hamilton directly. 
# Adding /results returns the results and stats on Hamilton's races
hamilton = '/drivers/hamilton/results.json'

# Limit lets us choose how much data we want our api to return
params = {'limit': 500}

# Combine the Base URL and our Racer's specifics
ham_url = f1_base + hamilton
ham_data = []
r = requests.get(ham_url, params = params)
if r.ok:
    ham_stats = r.json()
    # ham_stats['MRData']['RaceTable']['Races'] works through different tables/steps to get us to the right information
    ham_stats2 = ham_stats['MRData']['RaceTable']['Races']
    # The loop below gathers data from each race and adds it to lists which will be converted into our df
    for i in range(0, len(ham_stats2)):
        seasons = ham_stats2[i]
        race_stats = seasons['Results'][0]  
        ham_list = {'Season': seasons['season'],
                    'Race': seasons['raceName'],
                    'Team': race_stats['Constructor']['name'],
                    'Position': race_stats['position'],
                    'Starting Grid Position': race_stats['grid'],
                    'Time': race_stats['Time']['time'] if ('Time' in race_stats and 'time' in race_stats['Time']) else 'NA',
                    'Avg Speed': race_stats['FastestLap']['AverageSpeed']['speed'] if ('FastestLap' in race_stats and 'AverageSpeed' in race_stats['FastestLap'] and 'speed' in race_stats['FastestLap']['AverageSpeed']) else 'NA'}
        ham_data.append(ham_list)
    ham_df = pd.DataFrame(ham_data)
    # Replacing 'Grand Prix' with "GP" for simplicity :)
    ham_df['Race'] = ham_df['Race'].replace(' Grand Prix', ' GP', regex=True)
    ham_df.to_csv('hamilton_stats.csv', index = False)
    
  